3D人脸模型调研报告
=======================
吴凡

********

I.3D人脸模型技术概况
----------------------------

3D人脸模型技术的核心是3D人脸的重建，主要分为两个途径：

	（1），通过多摄像头或者多帧图像的关键点匹配(Stereo matching)，重建人脸的深度信息，从而得到模型
	（2），通过预先训练好的人脸模型(3d morphable model)，拟合单帧或多帧RGB图像或深度图像，从而得到3d人脸模型的个性化参数
	
（1）的特点是重建需要多摄像头，准确的重建需要calibration；或者连续帧图像，对连续帧的角度关系和人脸特点有一定要求。重建的3D模型可能不完整，需要后期补洞等处理。好处是重建比较迅速，对任意物体都可以重建出模型，且能反映物体的真实尺寸。  
（2）的特点是重建只需要一帧人脸图像，前提条件大大简化，但需要事先从大量3D人脸模型中训练出参数化的人脸模型，拟合人脸的过程可能找不到最符合实际情况的人脸，但始终能保证人脸的完整性，另外重建速度不如（1）快，3D模型的尺寸关系也不如（1）真实。

**********

II. 3D人脸模型在人脸识别领域的应用
------------------------------

在人脸识别领域，一个重要的问题就是解决人脸角度对识别准确率的影响，处理多角度人脸识别的几个**思路**是    

    （a）训练基于角度的分类器和人脸特征进行识别，即pose awared model，例如【5】中通过训练针对不同角度的cnn网络来提高人脸识别率  
    （b）对人脸进行旋正处理得到正面人脸，其中对人脸进行旋正主要是基于3d人脸模型，一种是利用固定的3d人脸模型(reference model)获得旋正后人脸的各部分的warping图像或直接render旋正过的3d人脸，另一种为采用3d morphable model(3DMM)进行3D人脸重建。
    
3d人脸模型在人脸识别领域的**应用**主要有：
	
#### （1）Pose Invariant Face Recognition.   
利用3d人脸对2d图像进行拟合，并进行face frontalization以及表情、光照移除，得到virtualized人脸正面图像，提高识别准确率，【1】中验证了利用固定的3d人脸模型进行face frontalization对人脸识别效果的提升，其他相关的论文还有【2】【3】【4】【15】【16】等。在【1】中，作者在USF Human ID 3D,  Multi-PIE, CMU-PIE, FERET, and FacePix(30)数据库上进行人脸识别测试，其中CMU-PIE和FERET数据库是针对各种人脸角度的常用数据库
![1](images/report/1.jpg)
【2】中作者总结了四种基于3D人脸实现pose robust face recognition的方式：
![2](images/report/2.png)

#### （2）Face Alignment. 
利用3d人脸检测大角度的人脸关键点或dense关键点，可参考论文【6】【8】，下图为3d dense face alignment与其他landmarks检测的对比
![2](images/report/3.png)  

#### （3）Data augmentation. 
3d人脸模型还用于生成大量的2d人脸图像，用于训练，如【1】【11】【12】等，【1】中作用通过3d人脸模型生成了19900张人脸图片，用于训练人脸关键点与人脸角度的regression模型，其人脸角度估算的准确度为
![2](images/report/4.png)  

***********

III. 3D Morphable Face Model技术实现
----------------------------------------

	利用参数化人脸模型拟合2d图像的做法被大量引用的论文最早是1999年的Volker Blanz的"A Morphable Model For The Synthesis Of 3D Faces"（论文【7】），目前3d人脸模型的重建的思路主要是在此论文思路之上的进一步应用或改进，在实时性上有很大的提升。Blanz基于3DMM做人脸识别的【13】里旋正效果演示：
![2](images/report/5.png)  

**3DMM**的实现的几个关键步骤为：

（1）采集大量人脸3D模型，通过optical flow或shape feature matching等方法获得人脸3D点云的点的一一对应关系，从而得到3D人脸的向量表示，其中可分为shape vector(x,y,z), texture vector(r,g,b)。  
（2）通过pca计算一组用于表示3d人脸线性空间的基向量，任意3d人脸可表达为一组基向量的参数。  
（3）基于任意2d人脸图像，通过拟合或者regression模型寻找最符合2d人脸图像的3d人脸模型。  
（4）对3d人脸模型进行旋正，获得virtualized正面人脸图像，进行识别。  


3D数据的收集和预处理是3DMM最耗时的一步，采集3D人脸模型的精确程度决定了后期重建3d人脸的精确程度，另外不同人种的脸型差异也会影响后期人脸重建的效果。除采集数据之外，还需要找到各个3d点云中点的一一对应关系。2006年Paysan et. Al在【9】中提供了一个公开的可供下载的3DMM-- Basel Face Model，基于100个男性和100女性人脸训练得到，图为BGM模型的演示效果
![2](images/report/6.png)  

2016年Huber et al.在【14】中公开了一个Surrey Face Model，论文中展示的效果如下：
![2](images/report/7.png)  


在根据2d人脸图像寻找最佳3d人脸模型这步中，3DMM提出时是采用了analysis-by-synthesis的方法迭代拟合出最佳3d模型，比较耗时，在【8】中作者采用cascaded regression的方法拟合3d人脸模型，获得了实时的效果，【6】中作者利用cnn训练regression模型寻找3d morphable model的参数，此外利用cascaded regresion拟合2d图像的还有【10】等。


*********

* Ref
-------------------------------

【1】A. Asthana, T. Marks, M. Jones, K. Tieu, and M. Rohith. Fully automatic pose-invariant face recognition via 3D pose normalization. In ICCV, 2011[[paper]](http://www.merl.com/publications/docs/TR2011-074.pdf)  
【2】D. Yi, Z. Lei, and S. Li. Towards pose robust face recognition. In CVPR, pages 3539–3545, 2013  
【3】T. Hassner, S. Harel, E. Paz, and R. Enbar. Effective face frontalization in unconstrained images. In CVPR, 2015  
【4】C. Ferrari, G. Lisanti, S. Berretti, and A. Del Bimbo. Dictionary learning based 3D morphable model construction for face recognition with varying expression and pose. In 3DV, 2015  
【5】Masi, I.; Rawls, S.; Medioni, G.; Natarajan, P. Pose-Aware Face Recognition in the Wild. In CVPR, 2016  
【6】X Zhu，Z Lei，X Liu，H Shi，SZ Li. Face Alignment Across Large Poses: A 3D Solution. In CVPR, 2015 [[paper]](http://www.cbsr.ia.ac.cn/users/hailinshi/papers/2016-cvpr-zhu/ARXIV2015_FaceAlignment.pdf)   
【7】Blanz, Volker , Vetter, Thomas. A Morphable Model For The Synthesis Of 3D Faces. In Conference on Computer Graphics & Interactive. 1999[[paper]](https://cseweb.ucsd.edu/~ravir/6998/papers/p187-blanz.pdf)  
【8】László A. Jeni1, Jeffrey F. Cohn1,2, and Takeo Kanade1. Dense 3D Face Alignment from 2D Videos in Real-Time. 2015 IEEE. Automatic Face and Gesture Recognition.  [[paper]](http://www.pitt.edu/~jeffcohn/biblio/Jeni15FG_ZFace.pdf)  
【9】 P Paysan. A 3D Face Model for Pose and Illumination Invariant Face Recognition. 2009 IEEE. [[paper]](http://gravis.dmi.unibas.ch/publications/2009/BFModel09.pdf) [[database&code]](http://faces.cs.unibas.ch/bfm/main.php?nav=1-0&id=basel_face_model)   
【10】P Huber. Fitting 3D Morphable Face Models using local features. ICIP, 2015 IEEE.  
【11】ZH Feng. Cascaded Collaborative Regression for Robust Facial Landmark Detection Trained using a Mixture of Synthetic and Real Images with Dynamic Weighting. 2015 IEEE.  
【12】M Ratsch. Wavelet Reduced Support Vector Regression for Efficient and Robust Head Pose Estimation. 2012 IEEE  
【13】Volker Blanz. Face Recognition based on a 3D Morphable Model. International Conference on Automatic Face & Gesture 2006[[paper]](http://mi.informatik.uni-siegen.de/publications/fg06_blanz.pdf)  
【14】Patrik Huber. A Multiresolution 3D Morphable Face Model and Fitting Framework. VISAPP 2016  [[paper]](http://epubs.surrey.ac.uk/809478/1/Huber-VISAPP-2016.pdf)[[github]](https://github.com/patrikhuber/eos)[[database]](http://www.cvssp.org/faceweb/3dmm/facemodel/)  
【15】Chu, B., Romdhani, S., Chen, L.3d-aided face recognition robust to expression and pose variations. In: IEEE Conference on Computer Vision and Pattern Recognition, CVPR 2014  
【16】Tena, J.R., Smith, R.S., Hamouz, M., Kittler, J., Hilton, A., Illingworth, J. 2D face pose normalisation using a 3D Morphable Model. In IEEE Conference on AVSS 2007  






The Latest
=================
_% To Do %_  
1. Automated 3D Face Reconstruction From Multiple Images Using Quality Measures, CVPR, 2016. [[Paper]](http://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Piotraschke_Automated_3D_Face_CVPR_2016_paper.pdf)  

2. A Robust Multilinear Model Learning Framework for 3D Faces, CVPR, 2016. [[Paper]](http://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Bolkart_A_Robust_Multilinear_CVPR_2016_paper.pdf)  

3. Adaptive 3D Face Reconstruction From Unconstrained Photo Collections, CVPR, 2016. [[Paper]](http://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Roth_Adaptive_3D_Face_CVPR_2016_paper.pdf)  

4. A 3D Morphable Model Learnt From 10,000 Faces, CVPR, 2016. [[Paper]](http://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Booth_A_3D_Morphable_CVPR_2016_paper.pdf)   